In [27]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords') # install NLTK data to home user directory
import unidecode
from gensim.models import Word2Vec
from ingredient_parser import ingredient_parser
import embeddings as emb
from sklearn.metrics.pairwise import cosine_similarity

# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ingredient.split()

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

def get_recommendations(N, scores,csv_file,lerning_param):
    """
    Top-N recomendations order by score
    """
    # load in recipe dataset
    df_recipes = pd.read_csv(csv_file)
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    lerning_param.insert(0, "title")
    lerning_param.append("score")
    recommendation = pd.DataFrame(columns=lerning_param)

    for i in top:
        # recommendation.at[i, "title"] = unidecode.unidecode(df_recipes["title"][i])
        for param in lerning_param[1:-1]:
            recommendation.at[i, param] = ingredient_parser_final(
                df_recipes[param][i]
            )
        recommendation.at[i, "score"] = f"{scores[i]}"
    return recommendation


In [29]:
#####
# TREINAR MODELO
#####

csv_file = "recipes.csv"

data = pd.read_csv(csv_file)
learning_param=["recipe"]#["title","ingredients","recipe"]
columns=data["recipe"]#data["title"] +','+ data["recipe"]
data["parsed"] = columns #.apply(ingredient_parser)

# data.head()

corpus = data["parsed"]#get_and_sort_corpus(data)
print(f"Length of corpus: {len(corpus)}")
        
model = Word2Vec(corpus, sg=0, workers=8, window=6, min_count=1, vector_size=100,compute_loss=True)

# Loss
print(model.get_latest_training_loss())

#Summarize vocabulary
words = list(model.wv.index_to_key)
words.sort()
len(words)

Length of corpus: 79
24938.642578125


62

In [37]:
#####
# TESTAR COM INPUT
#####
n_sugestions = 1
key_words = open("../test.txt", "r")#"francesinha"
input=key_words.read()
# vec_tr = emb.MeanEmbeddingVectorizer(model)
vec_tr = emb.TfidfEmbeddingVectorizer(model)
    
vec_tr.fit(corpus)
doc_vec = vec_tr.transform(corpus)
doc_vec = [doc.reshape(1, -1) for doc in doc_vec]

# create tokens with elements
#input = key_words.split(",")
# parse ingredient list
#input = ingredient_parser(input)
# get embeddings for ingredient doc
input_embedding = vec_tr.transform([input])[0].reshape(1, -1)

# get cosine similarity between input embedding and all the document embeddings
cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
scores = list(cos_sim)
# Filter top N recommendations
recommendations = get_recommendations(n_sugestions, scores,csv_file,learning_param)
# recommendations.recipe


ValueError: empty vocabulary; perhaps the documents only contain stop words